In [2]:
import torch

def load_file(path):
  with open(path) as f:
    return f.read().strip().split('\n\n')

In [4]:
load_file('example.txt')

['Monkey 0:\n  Starting items: 79, 98\n  Operation: new = old * 19\n  Test: divisible by 23\n    If true: throw to monkey 2\n    If false: throw to monkey 3',
 'Monkey 1:\n  Starting items: 54, 65, 75, 74\n  Operation: new = old + 6\n  Test: divisible by 19\n    If true: throw to monkey 2\n    If false: throw to monkey 0',
 'Monkey 2:\n  Starting items: 79, 60, 97\n  Operation: new = old * old\n  Test: divisible by 13\n    If true: throw to monkey 1\n    If false: throw to monkey 3',
 'Monkey 3:\n  Starting items: 74\n  Operation: new = old + 3\n  Test: divisible by 17\n    If true: throw to monkey 0\n    If false: throw to monkey 1']

In [33]:
def parse_monkey(raw_monkey):
  lines = raw_monkey.split('\n')

  id = lines[0].split(' ')[1]
  id = int(id[:-1])

  items = [float(item) for item in lines[1].split(':')[1].split(',')]

  operation = lines[2].split(':')[1].strip()
  operation = operation.replace('new = old ', '')
  [op, num] = operation.split(' ')
  def operation (x):
    y = x
    if num != 'old': 
      y = float(num)
    if op == '+': return x + y
    elif op == '*': return x * y
  
  test = lines[3:]
  # Right now, all the checks are modulo checks
  test_n = int(test[0].split('by ')[1])
  check_fn = lambda x: x % test_n == 0
  true_target = int(test[1].split('throw to monkey ')[1])
  false_target = int(test[2].split('throw to monkey ')[1])
  test = dict(check=check_fn, true_target=true_target, false_target=false_target)

  return dict(id=id, items=items, operation=operation, test=test, inspections=0, test_n=test_n)

In [6]:
parse_monkey(load_file('example.txt')[0])

{'id': 0,
 'items': [7.9, 9.8],
 'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
 'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
  'true_target': 2,
  'false_target': 3},
 'inspections': 0}

In [16]:
def get_monkeys(file):
  return [parse_monkey(raw_monkey) for raw_monkey in load_file(file)]

In [8]:
get_monkeys('example.txt')

[{'id': 0,
  'items': [8.5, 7.7, 7.7],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 6,
   'false_target': 7},
  'inspections': 0},
 {'id': 1,
  'items': [8.0, 9.9],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 3,
   'false_target': 5},
  'inspections': 0},
 {'id': 2,
  'items': [7.4, 6.0, 7.4, 6.3, 8.6, 9.2, 8.0],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 0,
   'false_target': 6},
  'inspections': 0},
 {'id': 3,
  'items': [7.1, 5.8, 9.3, 6.5, 8.0, 6.8, 5.4, 7.1],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 2,
   '

In [42]:
import math

def get_monkey_by_id(monkeys, id):
  for monkey in monkeys:
    if monkey['id'] == id:
      return monkey
      
def scale_monkeys(monkeys):
  scale = math.prod([monkey['test_n'] for monkey in monkeys])
  for monkey in monkeys:
    monkey['items'] = [item % scale for item in monkey['items']]
  return monkeys

def run_monkeys(monkeys, loop, log=True):
  for monkey in monkeys:
    if log: print('Monkey ' + str(monkey['id']))
    for item in monkey['items']:
      if log: print(f"  Monkey inspects an item with a worry level of {item}.")
      worry = monkey['operation'](item)
      if log: print(f"    Worry level is set to {worry}.")
      if log: print(f"    Monkey gets bored with item. Worry level is divided by 3 to {worry}.")
      if monkey['test']['check'](worry):
        if log: print(f"    Passed the monkey's check")
        if log: print(f"    Monkey throws item to monkey {monkey['test']['true_target']}.")
        target_monkey = get_monkey_by_id(monkeys, monkey['test']['true_target'])
        target_monkey['items'].append(worry)
      else:
        if log: print(f"    Failed the monkey's check")
        if log: print(f"    Monkey throws item to monkey {monkey['test']['false_target']}.")
        target_monkey = get_monkey_by_id(monkeys, monkey['test']['false_target'])
        target_monkey['items'].append(worry)
    monkey['inspections'] += len(monkey['items'])
    monkey['items'] = []
  if log: print(monkeys)
  monkeys = scale_monkeys(monkeys)
  return monkeys

In [48]:
def run_monkeys_n_times(monkeys, n):
  for i in range(n):
    monkeys = run_monkeys(monkeys, loop=i, log=False)
    iteration = i + 1
    if iteration % 1_000 == 0 or iteration == 20 or iteration == 1:
      print('Iteration:', i + 1)
      for monkey in monkeys:
        print(f"Monkey {monkey['id']} has inspected {monkey['inspections']} items.")
      print('')
  return monkeys

In [49]:
run_monkeys_n_times(get_monkeys('example.txt'), 10_000)

Iteration: 1
Monkey 0 has inspected 2 items.
Monkey 1 has inspected 4 items.
Monkey 2 has inspected 3 items.
Monkey 3 has inspected 6 items.

Iteration: 20
Monkey 0 has inspected 99 items.
Monkey 1 has inspected 97 items.
Monkey 2 has inspected 8 items.
Monkey 3 has inspected 103 items.

Iteration: 1000
Monkey 0 has inspected 5204 items.
Monkey 1 has inspected 4792 items.
Monkey 2 has inspected 199 items.
Monkey 3 has inspected 5192 items.

Iteration: 2000
Monkey 0 has inspected 10419 items.
Monkey 1 has inspected 9577 items.
Monkey 2 has inspected 392 items.
Monkey 3 has inspected 10391 items.

Iteration: 3000
Monkey 0 has inspected 15638 items.
Monkey 1 has inspected 14358 items.
Monkey 2 has inspected 587 items.
Monkey 3 has inspected 15593 items.

Iteration: 4000
Monkey 0 has inspected 20858 items.
Monkey 1 has inspected 19138 items.
Monkey 2 has inspected 780 items.
Monkey 3 has inspected 20797 items.

Iteration: 5000
Monkey 0 has inspected 26075 items.
Monkey 1 has inspected 2392

[{'id': 0,
  'items': [63602.0, 56040.0, 11941.0, 10573.0, 61607.0],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 2,
   'false_target': 3},
  'inspections': 52166,
  'test_n': 23},
 {'id': 1,
  'items': [90861.0, 86149.0, 27648.0, 21340.0, 76915.0],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 2,
   'false_target': 0},
  'inspections': 47830,
  'test_n': 19},
 {'id': 2,
  'items': [],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 1,
   'false_target': 3},
  'inspections': 1938,
  'test_n': 13},
 {'id': 3,
  'items': [],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_mo

In [51]:
[monkey['inspections'] for monkey in run_monkeys_n_times(get_monkeys('input.txt'), 10_000)]

Iteration: 1
Monkey 0 has inspected 3 items.
Monkey 1 has inspected 2 items.
Monkey 2 has inspected 7 items.
Monkey 3 has inspected 9 items.
Monkey 4 has inspected 14 items.
Monkey 5 has inspected 2 items.
Monkey 6 has inspected 11 items.
Monkey 7 has inspected 10 items.

Iteration: 20
Monkey 0 has inspected 119 items.
Monkey 1 has inspected 140 items.
Monkey 2 has inspected 147 items.
Monkey 3 has inspected 231 items.
Monkey 4 has inspected 158 items.
Monkey 5 has inspected 198 items.
Monkey 6 has inspected 135 items.
Monkey 7 has inspected 137 items.

Iteration: 1000
Monkey 0 has inspected 6002 items.
Monkey 1 has inspected 5692 items.
Monkey 2 has inspected 6353 items.
Monkey 3 has inspected 11668 items.
Monkey 4 has inspected 6034 items.
Monkey 5 has inspected 11628 items.
Monkey 6 has inspected 6010 items.
Monkey 7 has inspected 6343 items.

Iteration: 2000
Monkey 0 has inspected 12002 items.
Monkey 1 has inspected 11361 items.
Monkey 2 has inspected 12689 items.
Monkey 3 has insp

[60002, 56692, 63353, 116668, 60034, 116628, 60010, 63343]

In [52]:
116668 * 116628

13606755504

In [18]:
232 * 236

54752

In [ ]:
10_000 / 20

500.0

In [19]:
run_monkeys(get_monkeys('example.txt'), 0, log=True)

Monkey 0
  Monkey inspects an item with a worry level of 85.
    Worry level is set to 595.
    Monkey gets bored with item. Worry level is divided by 3 to 595.
    Failed the monkey's check
    Monkey throws item to monkey 7.
  Monkey inspects an item with a worry level of 77.
    Worry level is set to 539.
    Monkey gets bored with item. Worry level is divided by 3 to 539.
    Failed the monkey's check
    Monkey throws item to monkey 7.
  Monkey inspects an item with a worry level of 77.
    Worry level is set to 539.
    Monkey gets bored with item. Worry level is divided by 3 to 539.
    Failed the monkey's check
    Monkey throws item to monkey 7.
Monkey 1
  Monkey inspects an item with a worry level of 80.
    Worry level is set to 880.
    Monkey gets bored with item. Worry level is divided by 3 to 880.
    Failed the monkey's check
    Monkey throws item to monkey 5.
  Monkey inspects an item with a worry level of 99.
    Worry level is set to 1089.
    Monkey gets bored with

[{'id': 0,
  'items': [102, 61, 84, 63, 83, 77, 92, 66, 83, 1101],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 6,
   'false_target': 7},
  'inspections': 3},
 {'id': 1,
  'items': [9801,
   8100,
   7056,
   2500,
   6724,
   6724,
   5041,
   8836,
   10000,
   7744,
   53,
   69,
   95,
   67,
   81,
   4627],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 3,
   'false_target': 5},
  'inspections': 2},
 {'id': 2,
  'items': [70, 70, 105, 80],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,
  'test': {'check': <function __main__.parse_monkey.<locals>.<lambda>(x)>,
   'true_target': 0,
   'false_target': 6},
  'inspections': 7},
 {'id': 3,
  'items': [81, 884],
  'operation': <function __main__.parse_monkey.<locals>.operation(x)>,

In [17]:
[monkey['inspections'] for monkey in run_monkeys_n_times(get_monkeys('example.txt'), 1_000)]

Iteration: 0
Monkey 0 has inspected 2 items.
Monkey 1 has inspected 4 items.
Monkey 2 has inspected 3 items.
Monkey 3 has inspected 6 items.

Iteration: 100
Monkey 0 has inspected 502 items.
Monkey 1 has inspected 504 items.
Monkey 2 has inspected 3 items.
Monkey 3 has inspected 506 items.

Iteration: 200
Monkey 0 has inspected 996 items.
Monkey 1 has inspected 1010 items.
Monkey 2 has inspected 39 items.
Monkey 3 has inspected 1023 items.

Iteration: 300
Monkey 0 has inspected 1497 items.
Monkey 1 has inspected 1509 items.
Monkey 2 has inspected 49 items.
Monkey 3 has inspected 1529 items.

Iteration: 400
Monkey 0 has inspected 1997 items.
Monkey 1 has inspected 2009 items.
Monkey 2 has inspected 49 items.
Monkey 3 has inspected 2029 items.

Iteration: 500
Monkey 0 has inspected 2497 items.
Monkey 1 has inspected 2509 items.
Monkey 2 has inspected 49 items.
Monkey 3 has inspected 2529 items.

Iteration: 600
Monkey 0 has inspected 2997 items.
Monkey 1 has inspected 3009 items.
Monkey 2

[4992, 5004, 49, 5024]